In [1]:
from sklearn.datasets import load_iris

In [2]:
data = load_iris()

In [3]:
for i in data:
    print(i)

data
target
frame
target_names
DESCR
feature_names
filename
data_module


In [4]:
data['data']

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [5]:
data['target']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
print(len(data['data']),len(data['target']))

150 150


In [9]:
input = data['data']
labels = data['target']

In [10]:
print(type(input), type(labels))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [11]:
import torch

In [68]:
torch.cuda.is_available()

True

In [67]:
input = torch.FloatTensor(input)
labels = torch.LongTensor(labels)

input = input.cuda()
labels = labels.cuda()

In [13]:
print(type(input), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [21]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [24]:
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.fc1 = nn.Linear(4,9)
        self.fc2 = nn.Linear(9,5)
        self.fc3 = nn.Linear(5,10)
        self.fc4 = nn.Linear(10, 3)
           
    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

In [69]:
net = LeNet()
net = net.cuda()

In [26]:
res_test = net(input[0])
print(res_test)

tensor([-0.0405,  0.1438,  0.1334], grad_fn=<AddBackward0>)


In [29]:
print(input[:4,], labels[:4,])

tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000]]) tensor([0, 0, 0, 0])


In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [58]:
from tqdm import tqdm

In [70]:
for epoch in range(50):
    
    running_loss = 0.0
    
    for count in tqdm(range(30)):
        
        optimizer.zero_grad()
        
        output = net(input[count*5 : (count+1)*5 ])
        loss = criterion(output, labels[count*5 : (count+1)*5])
        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()
        
        if count%29 == 0:
            print('[%d,%5d] loss: %.3f' %
                 (epoch + 1, count+1, running_loss/29))
            
            running_loss = 0.0
print("done training")

 37%|██████████████████████████████                                                    | 11/30 [00:14<00:17,  1.06it/s]

[1,    1] loss: 0.043


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:14<00:00,  2.07it/s]


[1,   30] loss: 1.069


100%|█████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 333.33it/s]


...

[48,    1] loss: 0.043
[48,   30] loss: 1.069



100%|█████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 483.85it/s]

[49,    1] loss: 0.043
[49,   30] loss: 1.069



100%|█████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 454.54it/s]

[50,    1] loss: 0.043
[50,   30] loss: 1.069
done training


In [60]:
torch.save(net, 'course_0.pth')

In [65]:
res = net(input)
print(res)

tensor([[ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00

In [62]:
model = torch.load('course_0.pth')
model.eval()

LeNet(
  (fc1): Linear(in_features=4, out_features=9, bias=True)
  (fc2): Linear(in_features=9, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)

In [66]:
res = model(input)
print(res)

tensor([[ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00, -3.9628e-01, -1.8264e+00],
        [ 2.4948e+00